 数据集 BT-TWD 可行性实验

本 notebook 按步骤运行：加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD k 折实验 → 桶级分析。

In [1]:
# 步骤0：环境与路径设置
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    BucketTree,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = os.path.join(root_path, 'configs', 'synth_strong_v1.yaml')
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')

【INFO】【2025-12-16 16:34:10】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\synth_strong_v1.yaml
【INFO】【2025-12-16 16:34:14】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')

【INFO】【2025-12-16 16:34:15】【配置-数据】数据集=synth_strong_v1, k折=None, 目标列=target, 正类="1"
【INFO】【2025-12-16 16:34:15】【配置-BTTWD】阈值模式=bucket_wise, 全局模型=xgb, 桶内模型=none, 后验估计器(兼容字段)=logreg
【INFO】【2025-12-16 16:34:15】【配置-基线】LogReg启用=True, RandomForest启用=True, KNN启用=True, XGBoost启用=True
【INFO】【2025-12-16 16:34:15】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw, target_col_model = load_dataset(cfg)  # 这里返回的是用于建模的标签列，例如 "label"

display(df_raw.head())
print("用于建模的标签列:", target_col_model)

# 1）画 0/1 标签（延误/不延误）的比例
class_counts = df_raw[target_col_model].value_counts(normalize=True)
ax = class_counts.plot(kind='bar', title='正 vs 负比例')
plt.ylabel('比例')

fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()

# 2）如果想看原始 DepDelay 的分布，可以另外单独分析：
raw_target_col = cfg['DATA']['target_col']  # 这里是 "DepDelay"
print("原始目标列:", raw_target_col)
print(df_raw[raw_target_col].describe())

log_info('【步骤2摘要】Airlines 原始数据加载与基本统计完成。')


【INFO】【2025-12-16 16:34:15】【合成数据加载】文件=..\data\synth\synth_strong_v1.csv，样本数=200000，全局正例率=25.00%
【INFO】【2025-12-16 16:34:15】组别 A: 样本数=49807，正例率=2.43%
【INFO】【2025-12-16 16:34:15】组别 B: 样本数=49948，正例率=12.03%
【INFO】【2025-12-16 16:34:15】组别 C: 样本数=50323，正例率=26.81%
【INFO】【2025-12-16 16:34:15】组别 D: 样本数=49922，正例率=58.65%
【INFO】【2025-12-16 16:34:15】【数据集信息】名称=synth_strong_v1，样本数=200000，目标列=target，正类比例=25.00%


,target,group,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,z1,z2,z3,z4,z5
0,0,A,1.064588,-1.305676,0.557827,0.009463,2.368387,0.719543,0.007857,1.028664,0.582720,0.761297,1.054418,-0.801457,0.300131,-0.073666,-1.663032
1,1,D,0.512215,-0.375004,-0.383665,0.704062,-0.619332,0.181440,0.511407,-0.562699,0.456859,0.038571,0.511402,-0.505405,-1.402719,-0.680446,-0.082700
2,0,C,-0.370929,0.782064,-1.249109,-1.254847,0.056107,-1.111550,1.299804,-0.322134,-1.384439,-0.148576,-0.670660,0.892483,-0.741191,-1.742448,-0.781772
3,1,B,1.791067,-0.872657,-0.609798,0.565370,1.414436,-0.988554,-0.749953,-1.541402,-0.011446,0.252859,-0.328013,-0.757283,-0.919904,-1.548651,-1.364378
4,0,B,0.917348,0.433200,0.110524,0.265732,0.602970,-1.176482,0.260115,-1.655680,1.147569,1.809379,-0.471701,-0.053854,-0.310862,0.740968,1.703402


用于建模的标签列: target
原始目标列: target
count    200000.000000
mean          0.249955
std           0.432988
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: target, dtype: float64
【INFO】【2025-12-16 16:34:15】【步骤2摘要】Airlines 原始数据加载与基本统计完成。


In [4]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")

【INFO】【2025-12-16 16:34:16】【预处理】连续特征=15个，类别特征=1个
【INFO】【2025-12-16 16:34:16】【预处理】编码后维度=18
【INFO】【2025-12-16 16:34:16】【预处理】编码特征维度=18，样本数=200000
【INFO】【2025-12-16 16:34:16】【步骤3摘要】特征预处理完成：连续=15，类别=1，编码维度=18。


In [5]:
# 步骤4：构建桶树并检查划分
bucket_tree = BucketTree(cfg['BTTWD']['bucket_levels'], feature_names=df_raw.drop(columns=[cfg['DATA']['target_col']]).columns.tolist())
bucket_ids_full = bucket_tree.assign_buckets(df_raw.drop(columns=[cfg['DATA']['target_col']]))
bucket_df = bucket_ids_full.value_counts().reset_index()
bucket_df.columns = ['bucket_id', 'count']
bucket_df['pos_rate'] = df_raw.groupby(bucket_ids_full)[cfg['DATA']['target_col']].apply(lambda s: (s == cfg['DATA']['positive_label']).mean()).values
display(bucket_df.head())
bucket_df.set_index('bucket_id')['count'].plot(kind='bar', figsize=(12,4), title='桶样本数分布')
fig_bucket = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bucket_metrics_bar.png')
plt.savefig(fig_bucket, bbox_inches='tight')
plt.close()
log_info(f'【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。')

【INFO】【2025-12-16 16:34:16】【桶树】已为样本生成桶ID，共 12 个组合


,bucket_id,count,pos_rate
0,L1_group=C|L2_x1=mid,19213,0.016469
1,L1_group=D|L2_x1=mid,19111,0.033558
2,L1_group=A|L2_x1=mid,19066,0.023288
3,L1_group=B|L2_x1=mid,18937,0.090426
4,L1_group=B|L2_x1=low,15595,0.151138


【INFO】【2025-12-16 16:34:16】【步骤4摘要】桶树划分完成，共有 12 个叶子桶。


In [6]:
# 步骤5：运行基线模型 k 折实验
# 基线部分在 run_kfold_experiments 内统一调度
log_info('【步骤5】基线模型将在整体交叉验证中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')

【INFO】【2025-12-16 16:34:16】【步骤5】基线模型将在整体交叉验证中一并运行。
【INFO】【2025-12-16 16:34:16】【步骤5摘要】基线模型性能将作为后续对比基准。


In [7]:
import numpy as np
import pandas as pd

print("y 全局标签分布：", np.unique(y, return_counts=True))

print("原始 income 列分布：")
print(df_raw[cfg['DATA']['target_col']].value_counts())


y 全局标签分布： (array([0, 1]), array([150009,  49991], dtype=int64))
原始 income 列分布：
target
0    150009
1     49991
Name: count, dtype: int64


In [8]:
# 步骤6：运行 BTTWD k 折实验（含基线）
results = run_kfold_experiments(X, y, df_raw.drop(columns=[cfg['DATA']['target_col']]), cfg)
summary_df = pd.read_csv(os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'metrics_kfold_summary.csv'))
display(summary_df)
summary_df.plot(x='model', kind='bar', figsize=(8,4), title='模型指标对比')
fig_compare = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'metrics_compare.png')
plt.savefig(fig_compare, bbox_inches='tight')
plt.close()
log_info('【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。')

【INFO】【2025-12-16 16:34:16】【基线-LogReg】使用决策阈值=0.300（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:109: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
d:\Anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


【INFO】【2025-12-16 16:34:18】【基线-LogReg】整体指标：AUC_mean=0.949, AUC_std=0.001, BAC_mean=0.873, BAC_std=0.001, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.780, F1_std=0.003, Kappa_mean=0.697, Kappa_std=0.004, MCC_mean=0.703, MCC_std=0.004, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.712, Precision_std=0.005, Recall_mean=0.861, Recall_std=0.003, Regret_mean=0.226, Regret_std=0.003
【INFO】【2025-12-16 16:34:18】【基线-RF】使用决策阈值=0.300（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:109: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
d:\Anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


【INFO】【2025-12-16 16:37:32】【基线-RF】整体指标：AUC_mean=0.940, AUC_std=0.001, BAC_mean=0.865, BAC_std=0.001, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.760, F1_std=0.002, Kappa_mean=0.667, Kappa_std=0.003, MCC_mean=0.677, MCC_std=0.003, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.677, Precision_std=0.004, Recall_mean=0.868, Recall_std=0.003, Regret_mean=0.236, Regret_std=0.003
【INFO】【2025-12-16 16:37:32】【基线-KNN】使用决策阈值=0.300（fixed 模式）


...

【INFO】【2025-12-16 16:40:20】【阈值】桶 ROOT（n_val=9599）使用本地阈值 α=0.4000, β=0.2000
[INFO][BT][2025-12-16 16:40:20] 桶 bucket_id=ROOT level=0：
    n_train=22398, n_val=9599,
    BAC=0.805, F1=0.774, AUC=0.946,
    Regret=0.212, BND_ratio=0.103, POS_coverage=0.254,
    Score(f1_regret_bnd )=0.511
【INFO】【2025-12-16 16:40:21】【阈值】桶 L1_group=A 标记为弱桶，阈值将回退使用 ROOT 的阈值
[INFO][BT][2025-12-16 16:40:21] 桶 bucket_id=L1_group=A level=1：
    n_train=5554, n_val=2399,
    BAC=0.604, F1=0.337, AUC=0.938,
    Regret=0.080, BND_ratio=0.033, POS_coverage=0.012,
    Score(f1_regret_bnd )=0.241
[INFO][BT][2025-12-16 16:40:21] 桶 bucket_id=L1_group=A：
    parent_id=ROOT，parent_Score=0.511, bucket_Score=0.241,
    Gain=-0.270, is_weak=True
【INFO】【2025-12-16 16:40:21】【阈值】桶 L1_group=B 标记为弱桶，阈值将回退使用 ROOT 的阈值
[INFO][BT][2025-12-16 16:40:21] 桶 bucket_id=L1_group=B level=1：
    n_train=5591, n_val=2397,
    BAC=0.708, F1=0.584, AUC=0.913,
    Regret=0.204, BND_ratio=0.083, POS_coverage=0.089,
    Score(f1_regret_bnd )=0.338


C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [16:40:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-12-16 16:40:35】[BASELINE] 全局 XGB 模型训练完成
【INFO】【2025-12-16 16:40:35】[BASELINE] 阈值搜索开始
【INFO】【2025-12-16 16:40:38】[BASELINE] 最佳阈值找到: alpha=0.4000, beta=0.2000, regret=0.2121
【INFO】【2025-12-16 16:40:38】【桶树】已为样本生成桶ID，共 12 个组合
【INFO】【2025-12-16 16:40:38】[BASELINE] 测试集桶映射完成，共 12 个桶
【INFO】【2025-12-16 16:40:39】[BASELINE] 桶 L1_group=D|L2_x1=low: BAC=0.7737, Regret=0.2774, Precision=0.8126, Recall=0.9153
【INFO】【2025-12-16 16:40:39】[BASELINE] 桶 L1_group=A|L2_x1=mid: BAC=0.6435, Regret=0.0640, Precision=0.6410, Recall=0.2907
【INFO】【2025-12-16 16:40:39】[BASELINE] 桶 L1_group=C|L2_x1=mid: BAC=0.8006, Regret=0.3080, Precision=0.6933, Recall=0.7126
【INFO】【2025-12-16 16:40:39】[BASELINE] 桶 L1_group=D|L2_x1=high: BAC=0.7977, Regret=0.2988, Precision=0.7835, Recall=0.8910
【INFO】【2025-12-16 16:40:39】[BASELINE] 桶 L1_group=C|L2_x1=high: BAC=0.8159, Regret=0.2486, Precision=0.6968, Recall=0.7201
【INFO】【2025-12-16 16:40:39】[BASELINE] 桶 L1_group=C|L2_x1=low: BAC=0.8261, Regret=0.2901, Precision=0.749

,model,Precision_mean,Precision_std,Recall_mean,Recall_std,F1_mean,F1_std,BAC_mean,BAC_std,AUC_mean,...,MCC_mean,MCC_std,Kappa_mean,Kappa_std,BND_ratio_mean,BND_ratio_std,POS_Coverage_mean,POS_Coverage_std,Regret_mean,Regret_std
0,BTTWD,0.636488,0.004263,0.912824,0.003926,0.749996,0.001954,0.869531,0.000861,0.946834,...,0.667310,0.002279,0.645612,0.003330,0.11807,0.012754,0.272365,0.010259,0.210737,0.001658
1,LogReg,0.712163,0.004958,0.861355,0.002612,0.779673,0.002797,0.872658,0.001480,0.948969,...,0.702682,0.003802,0.696657,0.004188,0.00000,0.000000,NaN,NaN,0.225655,0.002578
2,RandomForest,0.676645,0.003738,0.867796,0.003346,0.760379,0.002055,0.864789,0.001318,0.939950,...,0.676891,0.002785,0.666778,0.003133,0.00000,0.000000,NaN,NaN,0.235850,0.002581
3,KNN,0.556321,0.003083,0.821288,0.002521,0.663317,0.002469,0.801497,0.001771,0.877044,...,0.541082,0.003538,0.520370,0.003838,0.00000,0.000000,NaN,NaN,0.342410,0.003107
4,XGBoost,0.711867,0.004288,0.856794,0.003476,0.777624,0.002646,0.870604,0.001716,0.947350,...,0.699780,0.003634,0.694094,0.003849,0.00000,0.000000,NaN,NaN,0.229875,0.003257


【INFO】【2025-12-16 16:40:39】【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。


In [9]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate_all', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标已整理，可用于局部化分析。')

,bucket_id,layer,parent_bucket_id,n_train,n_val,pos_rate_train,pos_rate_val,alpha,beta,regret_val,...,is_weak,threshold_source_bucket,parent_with_threshold,n_test,pos_rate_test,BND_ratio_test,POS_Coverage_test,regret_test,fold,pos_rate
0,ROOT,L1,NaN,22399,9599,0.250815,0.250755,0.3,0.1,0.210074,...,False,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.249956
1,L1_group=C,L1,ROOT,5588,2443,0.262527,0.267294,0.3,0.1,0.294924,...,True,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.269737
2,L1_group=B,L1,ROOT,5627,2380,0.120668,0.119328,0.3,0.1,0.205462,...,True,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.120877
3,L1_group=A,L1,ROOT,5602,2377,0.024813,0.023559,0.3,0.1,0.063736,...,True,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.024108
4,L1_group=D,L1,ROOT,5578,2396,0.577447,0.595576,0.3,0.1,0.292988,...,True,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.585628


【INFO】【2025-12-16 16:40:40】【步骤7摘要】桶级指标已整理，可用于局部化分析。


In [10]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['results_dir'])))
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['figs_dir'])))
log_info('【全部步骤完成】Adult 数据集上的 BT-TWD 可行性实验结束。')

【INFO】【2025-12-16 16:40:40】【步骤8】检查结果文件与图表。
['baseline_bucket_metrics.csv', 'bucket_fallback_stats.csv', 'bucket_metrics.csv', 'bucket_metrics_gain.csv', 'bucket_thresholds.csv', 'bucket_thresholds_per_fold.csv', 'bucket_tree_structure.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv', 'metrics_overview.csv']
['bank_class_distribution.png', 'bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-12-16 16:40:40】【全部步骤完成】Adult 数据集上的 BT-TWD 可行性实验结束。
